In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
df = pd.read_csv("sources/vehicles.csv", nrows=10000,index_col=0)
del df["url"]
del df["region_url"]
del df["image_url"]
del df["description"]
del df["VIN"]
del df["county"]
df.tail()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,lat,long,posting_date
id,,,,,,,,,,,,,,,,,,,
7316825923,phoenix,15999,2015.0,gmc,acadia sle-1,good,6 cylinders,gas,135000.0,clean,automatic,fwd,full-size,SUV,red,az,33.415592,-111.657979,2021-05-04T10:48:28-0700
7316825904,phoenix,6999,2012.0,ford,focus sel hatchback,good,4 cylinders,gas,153000.0,clean,automatic,fwd,mid-size,hatchback,silver,az,33.415592,-111.657937,2021-05-04T10:48:27-0700
7316825646,phoenix,24999,2015.0,ram,1500 big horn,good,8 cylinders,gas,107000.0,clean,automatic,rwd,full-size,pickup,silver,az,33.415592,-111.657915,2021-05-04T10:48:02-0700
7316825626,phoenix,7999,2011.0,ford,fusion sel,good,4 cylinders,gas,121000.0,clean,automatic,fwd,full-size,sedan,grey,az,33.415628,-111.657958,2021-05-04T10:48:00-0700
7316825372,phoenix,10999,2012.0,chrysler,200 s convertible,like new,6 cylinders,gas,94000.0,clean,automatic,fwd,mid-size,convertible,white,az,33.415628,-111.657894,2021-05-04T10:47:36-0700


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 7222695916 to 7316825372
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   region        10000 non-null  object 
 1   price         10000 non-null  int64  
 2   year          9887 non-null   float64
 3   manufacturer  9517 non-null   object 
 4   model         9871 non-null   object 
 5   condition     5347 non-null   object 
 6   cylinders     6713 non-null   object 
 7   fuel          9878 non-null   object 
 8   odometer      9942 non-null   float64
 9   title_status  9753 non-null   object 
 10  transmission  9917 non-null   object 
 11  drive         7101 non-null   object 
 12  size          2600 non-null   object 
 13  type          8149 non-null   object 
 14  paint_color   7984 non-null   object 
 15  state         10000 non-null  object 
 16  lat           9928 non-null   float64
 17  long          9928 non-null   float64
 18  posting_date  997

In [4]:
CAT_FEATURES = [ "year", "manufacturer", "model",  "condition", "cylinders","fuel","title_status","transmission", "drive", "size", "type", "paint_color"]
NUM_FEATURES = ["odometer"]

y = df["price"]                              

X = df[CAT_FEATURES + NUM_FEATURES]

for col in X[CAT_FEATURES].columns:
    print(f"{df[col].value_counts()}")

year
2017.0    973
2018.0    944
2019.0    851
2016.0    714
2020.0    703
         ... 
1926.0      1
1930.0      1
1922.0      1
1928.0      1
2022.0      1
Name: count, Length: 89, dtype: int64
manufacturer
ford               1710
chevrolet          1523
toyota              739
jeep                532
nissan              448
ram                 395
gmc                 390
dodge               384
honda               383
kia                 275
bmw                 271
subaru              206
mercedes-benz       198
lexus               192
hyundai             188
acura               175
volkswagen          164
cadillac            162
chrysler            137
audi                136
buick               124
infiniti            120
lincoln             105
mazda                95
volvo                77
mitsubishi           55
jaguar               54
rover                51
mini                 51
pontiac              42
alfa-romeo           34
porsche              22
tesla                2